In [1]:
import numpy as np

# 1.1. Fancy indexing

You know already how to access single items in an array and how to slice an array. 
For example for the following array ```a```


In [32]:
a = np.arange(12).reshape((4,3))
print(a)

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]


In [39]:
# to get a single element you would do...

# to get a *slice* of the array you could do...


There is another way of indexing: when you pass multiple lists. 
They are interpreted as **coordinates to look up**. This is *fancy-indexing*.

In [40]:
a[[0,1],[1,2]]

array([1, 5])

Another example

In [6]:
a = np.array(list("ABCDEFGHIJKLMNOPQRSTUVWXYZ1234")). reshape (5,6)
a

array([['A', 'B', 'C', 'D', 'E', 'F'],
       ['G', 'H', 'I', 'J', 'K', 'L'],
       ['M', 'N', 'O', 'P', 'Q', 'R'],
       ['S', 'T', 'U', 'V', 'W', 'X'],
       ['Y', 'Z', '1', '2', '3', '4']], dtype='<U1')

In [7]:
print (a[[2, 2, 1] ,[2 ,0 ,0]])

['O' 'M' 'G']


<div>
<img src="fancy_indexing_lookup.png" width="800" align='center'/>
</div>

In [8]:
rows = np.array([[2, 2, 1], [3, 2, 3]])
cols = np.array([[2, 0, 0], [4, 2, 4]])
print(rows)
print()
print(cols)
print()
print(a[rows, cols])

[[2 2 1]
 [3 2 3]]

[[2 0 0]
 [4 2 4]]

[['O' 'M' 'G']
 ['W' 'O' 'W']]


### You can also mix fancy indexing with "normal" indexing or slicing

### Fancy indexing also work with booleans

### Masking

Alternatively you can use the argument ```mask``` to select and operate on only a subset of the array

# 1.2 Views and Copies: an important distinction!


**View**

- accessing the array without changing the databuffer 
- *simple indexing* and *slicing* give views
- *in-place* operations can be done in views


**Copy**
- when a new array is created by duplicating the data buffer as well as the array metadata
- *fancy indexing* give always copies
- a copy can be forced by method .copy()

How to know? with ```base```

In [9]:
a = np.arange(1,6)
print(a)
v = a[2:5] # v is a view
print(v)
print(v.base)  # returns the 'base' array

[1 2 3 4 5]
[3 4 5]
[1 2 3 4 5]


In [10]:
a = np.arange(1,6)
print(a)
w = a.copy()
w.base  # returns None
w.base==None

[1 2 3 4 5]


True

As a copy is a different array in memory, modifiying it will *not* change the base array

In [11]:
a = np.arange(1, 6)
print('a:', a)
v = a[[1,2]]
print('v:', v)

a: [1 2 3 4 5]
v: [2 3]


In [12]:
v[0] = 99
print('v:', v)
print('a:', a)

v: [99  3]
a: [1 2 3 4 5]


The same operation with a *view*, however, will carry the change 

In [13]:
v = a[1:3]
v[0] = 99
print('v:', v)
print('a:', a)

v: [99  3]
a: [ 1 99  3  4  5]


# 1.3 Sorting

# Exercises on indexing, fancy indexing, views/copies and sorting


### Exercise 1


---

# Optional advanced topic: strides

The reason why some indexing gives *views* and other give *copies* lays in how numpy arranges the data in memory.

When you create an array, numpy allocates certain memory that depends on the type you choose


In [14]:
a = np.arange(9).reshape(3,3)
print(a)
a.dtype

[[0 1 2]
 [3 4 5]
 [6 7 8]]


dtype('int64')

In [15]:
a.itemsize

8

In this example the array has 8 bytes allocated per item.

Memory is *linear*, that means, the 2-D array will look in memory something like this (blue boxes) 

![memory.png](memory.png)

However, the user 'sees' the array in 2D. 

How does numpy accomplishes this? By defining ```strides```.


In [16]:
a.strides

(24, 8)

Strides tell you by how many bytes you should move in memory when moving one step in that dimension. This is better explain with the following visualization

![strides2.png](strides2.png)

In the example, to go from the first item in the first row to the first item in the second row, you need to move 24 bytes. To move from the column-wise, you just need to move 8 bytes.

**Views** are created when you just use other strides to read your data. Slicing and regular indexing allows that, as you know how many byte steps you need to take to get the data.

**Fancy indexing** does not allow that, because the data you are asking **cannot** be obtained by just changing the strides. Thus, numpy need to make a **copy** of it in memory.

Now, you can change the strides of an array at will.

In [17]:
a.strides=(8,24)

In [18]:
a

array([[0, 3, 6],
       [1, 4, 7],
       [2, 5, 8]])

By swapping the stride numbers we actually transposed the matrix.

We could do windows to the data by changing to

In [22]:
a.strides=(8, 8)
a

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

 But be careful! Changing the strides to something non-sensical will also give you non-sense. And numpy will not complain. 

In [20]:
a.strides=(8, 9)

In [21]:
a

array([[                 0, 144115188075855872,    844424930131968],
       [                 1, 216172782113783808,   1125899906842624],
       [                 2, 288230376151711744,   1407374883553280]])

Further resources on strides: 
- https://scipy-lectures.org/advanced/advanced_numpy/#indexing-scheme-strides
- https://ajcr.net/stride-guide-part-1/